<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Tutorial de Pytorch 10:  Transfer Learning**



In [2]:
import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch import nn, optim

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Cargamos el dataset completo
full_dataset = ImageFolder(root='data/house_rooms', transform=transform)

# Dividimos el dataset en entrenamiento y validación
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Cargamos la VGG16 preentrenada
vgg16 = models.vgg16(pretrained=True)

# Mantenemos solo las capas convolutivas de VGG16
features = vgg16.features

# Creamos una nueva sección de clasificación
classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512 * 7 * 7, 4096),  # Ajustamos el tamaño de entrada según el tamaño de salida de las capas convolutivas
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 5)  # 5 categorías
)

# Combinamos las capas convolutivas con la nueva sección de clasificación
model = nn.Sequential(
    features,
    classifier
)

# Movemos el modelo a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Criterio y Optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento y Validación
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Modo entrenamiento
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Época {epoch+1}, Pérdida de Entrenamiento: {round(running_loss/len(train_loader),3)}")
    
    # Evaluación con el conjunto de validación
    model.eval()  # Modo evaluación
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Época {epoch+1}, Pérdida de Validación: {round(val_loss/len(val_loader),3)}")
    print(f"Época {epoch+1}, Precisión de Validación: {round(100 * correct / total,3)}%")
    print("--------------------------------------------------")


Época 1, Pérdida de Entrenamiento: 1.7727527925462434
Época 1, Pérdida de Validación: 1.5888815468007869
Época 1, Precisión de Validación: 24.571428571428573%
--------------------------------------------------
Época 2, Pérdida de Entrenamiento: 1.5775126173640743
Época 2, Pérdida de Validación: 1.5896912271326238
Época 2, Precisión de Validación: 24.476190476190474%
--------------------------------------------------


KeyboardInterrupt: 